**NAIVE BAYES**

Baseado no teorema de bayes (análise probabilistica)

Aplicações mais comuns :

- Detecção de SPAN
- Detecção de emoções em frases
- Separação de documentos

O naive bayes a partir da tabela de dados previsores gera uma tabela de probabilidades que é usada como base para classificar novos dados.

Vantagens :

- Rápido se comparado a abordagens mais complexas ( Ex: redes neurais )
- Simples
- Capaz de tabalhar com altas dimensões (atributos)
- Boas previsões em bases de dados pequenas ( 400 - 1000 registros)

Desvantagens :
- Presume que os atributos previsores são totalmente independentes, o que nem sempre é verdade.



In [27]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


# carregando a base de dados de censo
base = pd.read_csv('../../res/census.csv')
base.shape


(32561, 15)

In [28]:
# separando os dados de previsao e classificacao
previsores = base.iloc[:, 0:14].values
classificadores = base.iloc[:, 14].values




#efetuando correcoes nos dados do censo

#transformando dados categorios da base em dados discretos

labelencoder_prev = LabelEncoder()
previsores[:, 1] = labelencoder_prev.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_prev.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_prev.fit_transform(previsores[:, 5])
previsores[:, 6] = labelencoder_prev.fit_transform(previsores[:, 6])
previsores[:, 7] = labelencoder_prev.fit_transform(previsores[:, 7])
previsores[:, 8] = labelencoder_prev.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_prev.fit_transform(previsores[:, 9])
previsores[:, 13] = labelencoder_prev.fit_transform(previsores[:, 13])


from sklearn.compose import make_column_transformer
preprocess = make_column_transformer(( OneHotEncoder(categories='auto'), [1,3,5,6,7,8,9,13] ),remainder="passthrough") 
previsores = preprocess.fit_transform(previsores).toarray()

#print(previsores)
#deprecated
#onehotencoder = OneHotEncoder(categorical_features = [1,3,5,6,7,8,9,13])
#previsores = onehotencoder.fit_transform(previsores).toarray()
#print(previsores)


In [29]:
# padronizando os valores dos previsores ( nao deve ser feito para todos os parametros sob risco de degradar a performance do algoritimo)
scaler = StandardScaler()
previsores_escalonados = scaler.fit_transform(previsores)



#separando os valores de teste e treinamento 
previsores_escalonados_treinamento, previsores_escalonados_teste, classificadores_treinamento, classificadores_teste = train_test_split(previsores_escalonados, classificadores, test_size=0.15, random_state=0)

print ("previsores escalonados treinamento /teste ",previsores_escalonados_treinamento.shape,previsores_escalonados_teste.shape)
print ("classificadores treinamento /teste ",classificadores_treinamento.shape,classificadores_teste.shape)

previsores escalonados treinamento /teste  (27676, 108) (4885, 108)
classificadores treinamento /teste  (27676,) (4885,)


In [30]:
# instanciando o naive bayes com o scikit
classificador = GaussianNB()
classificador.fit(previsores_escalonados_treinamento, classificadores_treinamento)

# rodando previsoes com o dado de teste
previsoes_dados_escalonados = classificador.predict(previsores_escalonados_teste)

In [31]:
#testes dessa instancia algoritimo

# o dado de precisao per se nao quer dizer muita coisa e preciso verificar outras metricas
precisao = accuracy_score(classificadores_teste, previsoes_dados_escalonados)

# uma dessas metricas eh a matriz de confusao ... ela e capaz de mostrar o desempenho do algoritimo para cada classe 
matriz = confusion_matrix(classificadores_teste, previsoes_dados_escalonados)

#o scikit tambem possui uma classe utilitaria que prove um report mais detalhado...
report = classification_report(classificadores_teste, previsoes_dados_escalonados)

print("Precisão:\n")
print(precisao)
print("\nMatriz de confusão:\n")
print(matriz)
print("\nReport:\n")
print (report)

Precisão:

0.4767656090071648

Matriz de confusão:

[[1172 2521]
 [  35 1157]]

Report:

              precision    recall  f1-score   support

       <=50K       0.97      0.32      0.48      3693
        >50K       0.31      0.97      0.48      1192

    accuracy                           0.48      4885
   macro avg       0.64      0.64      0.48      4885
weighted avg       0.81      0.48      0.48      4885

